In [20]:
# create studentroll-name-allocation csv fimax_matching_datae-mappings.csv

#from f1.csv
import pandas as pd

# Define the input and output fimax_matching_datae names
input1 = "final_allocation_1.csv"
input2 = "final_allocation_2.csv"
input3 = "final_allocation_3.csv"
input4 = "final_allocation_4.csv"
input5 = "final_allocation_5.csv"
output_filename = "final_mappings_lastyear.csv"


# Read the input CSV file into a pandas DataFrame
input_df1 = pd.read_csv(input1)
input_df2 = pd.read_csv(input2)
input_df3 = pd.read_csv(input3)
input_df4 = pd.read_csv(input4)
input_df5 = pd.read_csv(input5)


# Extract specific columns (Roll No, Name of Student, Allocation)
columns_to_extract = ["Roll No", "Name of Student", "Allocation"]
output_df1 = input_df1[columns_to_extract]
output_df2 = input_df2[columns_to_extract]
output_df3 = input_df3[columns_to_extract]
output_df4 = input_df4[input_df4["Eligible for TAship"] == "Yes"][columns_to_extract]
output_df5 = input_df5[input_df5["Eligible for TAship"] == "Yes"][columns_to_extract]


# Write the extracted data to the output CSV file
output_df1.to_csv(output_filename, index=False)
output_df2.to_csv(output_filename, mode='a', header=False, index=False)
output_df3.to_csv(output_filename, mode='a', header=False, index=False)
output_df4.to_csv(output_filename, mode='a', header=False, index=False)
output_df5.to_csv(output_filename, mode='a', header=False, index=False)







In [21]:
import pandas as pd
from fuzzywuzzy import fuzz

# Function to calculate the percent match between two names
def name_similarity(name1, name2):
    return fuzz.ratio(name1, name2)

# Read the first CSV file
df1 = pd.read_csv('final_mappings_lastyear.csv')

# Create a mapping of names to Roll No from the first CSV file
name_to_roll = {row['Name of Student']: row['Roll No'] for index, row in df1.iterrows()}



# Read the second CSV file
df2 = pd.read_csv('courses_info_name.csv')
count_no_of_courses=0 #
# # Process the data from the second CSV file
result_data = []
for index, row in df2.iterrows():
    course = row['Course']
    ta = row['TA']
    names = row['Names']
    matching_roll_no = None
    if not pd.isna(names) and names != "None":
        names = names.split('-')
        count_no_of_courses+=1
        for name in names:
            name = name.strip()  # Remove leading/trailing spaces
            best_match_score=0
            best_match_name=None
            for mapped_name in name_to_roll:
                similarity_score = name_similarity(name, mapped_name)
                if similarity_score > best_match_score:
                    best_match_name = mapped_name
                    best_match_score = similarity_score
            if best_match_name is not None:
                matching_roll_no = name_to_roll[best_match_name]
            result_data.append([course,ta,matching_roll_no])


# Create a DataFrame for the result
result_df = pd.DataFrame(result_data, columns=['Course', 'TA', 'Roll No'])
print(count_no_of_courses)
# Write the result to a third CSV file
result_df.to_csv('course_requests_info_roll.csv', index=False)


25


In [22]:
## Verification of Code Maximum Matching
import numpy as np
from my_matching import *
import networkx as nx
n=3
matrix = np.random.randint(50, 91, (n, n))
np.fill_diagonal(matrix, 100)
# for col in range(n):
#     np.random.shuffle(matrix[:, col])
matrix_dict = {}
vacancies={}
for row_number, row_values in enumerate(matrix):
    row_dict = {f'S{col+1}': value for col, value in enumerate(row_values)}
    matrix_dict[f'C{row_number+1}'] = row_dict
    vacancies[f'C{row_number+1}']=1

match,max_weight=matching(matrix_dict,vacancies)
print(match)
print(max_weight)
# print(vacancies)


{('S1', 'C1_0'), ('S3', 'C3_0'), ('S2', 'C2_0')}
300


In [23]:
df = pd.read_csv('course_requests_info_roll.csv')

# Create a dictionary with course names as keys and lists of roll numbers as values
course_imp = {}
for index, row in df.iterrows():
    course = row['Course']
    roll_no = row['Roll No']
    
    if course in course_imp:
        course_imp[course].append(roll_no)
    else:
        course_imp[course] = [roll_no]
df1= pd.read_csv('courses_info_name.csv')
for index, row in df1.iterrows():
    course=row['Course']
    if course not in course_imp:
        course_imp[course]=[]

print(course_imp['CS101'])

['194050015', '22m0760', '22M0783', '22M0750', '22M0757']


In [24]:
import random
random.seed(0)
df1=pd.read_csv('final_mappings_lastyear.csv')
df2=pd.read_csv('courses_info_name.csv')
rolls=list(df1['Roll No'])
courses=list(df2['Course'])
vacancies=list(df2['TA'])
course_vacancies={}
course_preferences={}
for i in range(len(courses)):
    course_vacancies[courses[i]]=vacancies[i]

for c in courses:
    dict_for_course_c={}
    count=course_vacancies[c]*100
    m=0
    for j in rolls:
        dict_for_course_c[j]=0
        if j in course_imp[c]:
            m+=1
    n=course_vacancies[c]-m
    if n<0:
        for k in course_imp[c]:
            dict_for_course_c[k]=count/m
    elif n==0 or m==0:
        for k in rolls:
            dict_for_course_c[k]=100
    else:
        a=100-20*(m/n)
        if a<80:
            a=80
        # print(m,n,a,c)
        random_number = random.randint(int(a), 100)
        for k in rolls:
            if k in course_imp[c]:
                dict_for_course_c[k]=100+(100-random_number)*(n/m)
            else:
                dict_for_course_c[k]=random_number
    course_preferences[c]=dict_for_course_c



In [25]:
from my_matching import *
m,w=matching(course_preferences,course_vacancies)

print(m)
import pandas as pd
alloted_weight=0
# Assuming the CSV file is named 'your_file.csv'
df = pd.read_csv('final_mappings_lastyear.csv')

for index, row in df.iterrows():
    course = row['Allocation']
    if course in courses:
        roll_no = row['Roll No']
        alloted_weight+=course_preferences[course][roll_no]
    # print(f"Course: {course}, Roll No: {roll_no}")
print(alloted_weight)



{('214050005', 'CS152_8'), ('23M0778', 'CS310_3'), ('204050003', 'DeptWork_2'), ('22D0377', 'CS154_1'), ('23M0801', 'CS207_2'), ('214050007', 'CS154_3'), ('23M0814', 'CS230_10'), ('22M0788', 'CS101_51'), ('214050006', 'CS152_9'), ('22M0765', 'CS101_18'), ('22M0807', 'CS337_3'), ('23M0769', 'CS310_7'), ('22M0813', 'CS101_31'), ('CS101_0', '23M2101'), ('22M0806', 'CS101_48'), ('22M0792', 'CS101_27'), ('23M0795', 'CS215_4'), ('23M2105', 'CS101_65'), ('23M0748', 'CS663_2'), ('23D0362', 'CS154_12'), ('23M0765', 'CS224M_3'), ('22M0766', 'CS725_2'), ('23M0799', 'CS207_3'), ('22M0798', 'CS101_12'), ('22m0760', 'CS101_3'), ('22m0772', 'CS101_36'), ('194052001', 'DeptWork_1'), ('23M0786', 'CS232_11'), ('23M0768', 'CS215_7'), ('23M0810', 'CS232_0'), ('204050001', 'CS152_0'), ('22M0779', 'CS725_3'), ('21Q050010', 'CS101_22'), ('21Q050001', 'CS101_58'), ('22M0752', 'CS347_4'), ('23M0806', 'CS207_5'), ('22M0755', 'CS101_9'), ('23M0779', 'CS310_2'), ('22M2108', 'CS101_11'), ('23M0747', 'CS218M_2'), (

In [26]:
max_matching_data=[] # strictly in the format roll,allocation
for t in m:
    if t[0][0]!='1' and t[0][0]!='2':
        max_matching_data.append([t[0],t[1]])
    else:
        max_matching_data.append([t[1],t[0]])


# Remove the part after the underscore in course names
data = [[course.split('_')[0], course_code] for course, course_code in max_matching_data]

# Create a DataFrame from the modified data
df = pd.DataFrame(data, columns=['Course', 'Roll No'])

# Write the DataFrame to a CSV file
csv_file = 'output_by_maximum_matching.csv'
df.to_csv(csv_file, index=False)
        

In [27]:

import pandas as pd
df1=pd.read_csv("course_requests_info_roll.csv")
df2=pd.read_csv("output_by_maximum_matching.csv")
df3=pd.read_csv("final_mappings_lastyear.csv")
courses_list=list(set(df1["Course"]))
rolls=list(df1['Roll No'])
total_data=[]
requests=0
alloted_before=0
alloted_matching=0
for c in courses_list:
    count1=0
    count2=0
    for r in course_imp[c]:
        if df1[df1['Roll No']==r].iloc[0,0] == df2[df2['Roll No'] == r].iloc[0,0]:
            count1+=1
        if df1[df1['Roll No']==r].iloc[0,0] == df3[df3['Roll No'] == r].iloc[0,2]:
            count2+=1
    requests+=len(course_imp[c])
    alloted_before+=count2
    alloted_matching+=count1
    total_data.append([c,len(course_imp[c]),count2,count1])
    
    # print(f"Course name:{c} Requests:{len(course_imp[c])} Originally Allocated:{count2} Allocated By Matching:{count1}\n")

result_df = pd.DataFrame(total_data, columns=['Course', 'No of Requests', 'Originally Allocated last year','Allocated by Maxium Matching'])

result_df.to_csv('analysis.csv', index=False)
print(requests)
print(alloted_before)
print(alloted_matching)


118
60
69


In [28]:
c='CS224M'
df1 = pd.read_csv('course_requests_info_roll.csv')
df2 = pd.read_csv('output_by_maximum_matching.csv')
rolls = list(df1[df1['Course'] ==c]['Roll No'])

for i in rolls:
    s = df2[df2['Roll No'] == i]['Course']
    
    if not s.empty and s.iloc[0] == c:  # Check if s is not empty and the course is 'CS101'
        print("Roll No:", i, "Allocated")
    else:
        w2 = course_preferences[s.iloc[0]][i]
        w1 = course_preferences[c][i]
        print("Roll No:", i, "Not Allocated, Given weight", w1)
        print(f"Allocated to {s.iloc[0]} weight", w2)


Roll No: 214050006 Not Allocated, Given weight 100
Allocated to CS152 weight 100
Roll No: 194050006 Not Allocated, Given weight 100
Allocated to CS213 weight 120.0
Roll No: 22M2106 Not Allocated, Given weight 100
Allocated to CS6001 weight 105.33333333333333
Roll No: 22M0801 Not Allocated, Given weight 100
Allocated to CS337 weight 105.0
Roll No: 22M0815 Not Allocated, Given weight 100
Allocated to CS101 weight 99
Roll No: 23M2108 Not Allocated, Given weight 100
Allocated to CS101 weight 99
